In [14]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [15]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [17]:
# 输入研究区的shp文件所在路径
wuhan_shp ="D:/NDVI_exercise/wuhan/wuhanregion.shpe"
# shp 转 ee 
wuhan_ee = geemap.shp_to_ee(wuhan_shp)
# 添加到layer
# Map.addLayer(wuhan_ee,{},"wuhan1+8")
Map.centerObject(wuhan_ee,7)   # 设置以研究区居中

In [18]:
JRC_permanent_water = []

# landsat 8 可用年份为2014-2020

In [19]:
# 去云掩膜函数
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
  qa = image.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

# 可视化参数设置
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

def NDVI(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(mndwi)

# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# 分类标签
label = 'waterclass'

In [24]:
# 可用年份为2014-2020
Year = 2015
month = 4
# 定义一些变量
days_31 = [1,3,5,7,8,10,12]
days_30 = [4,6,9,11]
days_28 = [2]
for month in range(6,13,1):
    if month in days_31:
        landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(wuhan_ee) \
            .filterDate('{}-{}-01'.format(Year,month), '{}-{}-31'.format(Year,month)) \
            .map(maskL8sr) \
            .map(NDVI) \
            .map(mNDWI)
    if month in days_30:
        landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(wuhan_ee) \
            .filterDate('{}-{}-01'.format(Year,month), '{}-{}-30'.format(Year,month)) \
            .map(maskL8sr) \
            .map(NDVI) \
            .map(mNDWI)
    if month in days_28:
        landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(wuhan_ee) \
            .filterDate('{}-{}-01'.format(Year,month), '{}-{}-28'.format(Year,month)) \
            .map(maskL8sr) \
            .map(NDVI) \
            .map(mNDWI)
    #合并
    wuhan_landsat8 = landsat_images.qualityMosaic('NDVI').clip(wuhan_ee)
    Map.addLayer(wuhan_landsat8,vis_params,"wuhan")
    # 当mNDWI > EVI时
    wuhan_mNDWI_NDVI = wuhan_landsat8.select('mNDWI').gt(wuhan_landsat8.select('NDVI'))
    wuhan_water = wuhan_mNDWI_NDVI.select('mNDWI').rename('waterclass')

    # Make the training dataset.
    points = wuhan_water.sample(**{
        'region': wuhan_ee,
        'scale': 30,
        'numPixels': 1000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })

    # Overlay the points on the imagery to get training.
    sample_point = wuhan_landsat8.select(bands).sampleRegions(**{
        'collection': points,
        'properties': [label],
        'scale': 30
    })

    #  在training要素集中增加一个random属性，值为0到1的随机数
    withRandom = sample_point.randomColumn('random')

    #把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
    split = 0.7
    trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
    testingPartition = withRandom.filter(ee.Filter.gte('random', split))

    # 利用70%数据随机森林进行训练
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(trainingPartition,label,bands)
    #对Landsat-8进行分类
    result=wuhan_landsat8.select(bands).classify(trainedClassifier)
    Map.addLayer(result,{'min':0,'max':1,'palette': ['white','blue']},'RF_water')

    water = result.eq(1).selfMask().multiply(ee.Image.pixelArea()).divide(1e6)
    areas = water.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': wuhan_ee.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    JRC_permanent_water.append(areas.get('classification').getInfo())
    print('{} month:{}'.format(month,areas.get('classification').getInfo()))

# train_accuracy = trainedClassifier.confusionMatrix()
# print('accuracy:',train_accuracy.accuracy().getInfo())
# print('kappa:',train_accuracy.kappa().getInfo())
# print('producersAccuracy:',train_accuracy.producersAccuracy().getInfo())
# print('consumersAccuracy:',train_accuracy.consumersAccuracy().getInfo())

# validated = testingPartition.classify(trainedClassifier)

# test_accuracy = validated.errorMatrix('waterclass', 'classification')

# print('accuracy:',test_accuracy.accuracy().getInfo())
# print('kappa:',test_accuracy.kappa().getInfo())
# print('producersAccuracy:',test_accuracy.producersAccuracy().getInfo())
# print('consumersAccuracy:',test_accuracy.consumersAccuracy().getInfo())

6 month:462.3357181844747
7 month:803.3744604714041
8 month:954.7123528474002
9 month:817.0491499024548
10 month:1039.7974312507413
11 month:1126.3098211173071


EEException: No data was found in classifier training input.

# landsat 7 可用年份为1999-2020

In [10]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

# 可视化参数设置
vis_params = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

def NDVI(image):
    ndvi=image.normalizedDifference(['B4', 'B3']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B2', 'B5']).rename("mNDWI")
    return image.addBands(mndwi)

# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
# 分类标签
label = 'waterclass'

In [12]:
# 可用年份为1999-2020
year = 2011
days_31 = [1,3,5,7,8,10,12]
days_30 = [4,6,9,11]
days_28 = [2]
for month in range(8,13,1):
    if month in days_31:
        landsat_images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
            .filterBounds(wuhan_ee) \
            .filterDate('{}-{}-01'.format(year,month), '{}-{}-31'.format(year,month)) \
            .map(cloudMaskL457) \
            .map(NDVI) \
            .map(mNDWI)
    if month in days_30:
        landsat_images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
            .filterBounds(wuhan_ee) \
            .filterDate('{}-{}-01'.format(year,month), '{}-{}-30'.format(year,month)) \
            .map(cloudMaskL457) \
            .map(NDVI) \
            .map(mNDWI)
    if month in days_28:
        landsat_images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
            .filterBounds(wuhan_ee) \
            .filterDate('{}-{}-01'.format(year,month), '{}-{}-28'.format(year,month)) \
            .map(cloudMaskL457) \
            .map(NDVI) \
            .map(mNDWI)
    #合并
    wuhan_landsat7 = landsat_images.qualityMosaic('NDVI').clip(wuhan_ee)
    # 当mNDWI > EVI时
    wuhan_mNDWI_NDVI = wuhan_landsat7.select('mNDWI').gt(wuhan_landsat7.select('NDVI'))
    wuhan_water = wuhan_mNDWI_NDVI.select('mNDWI').rename('waterclass')

    # Make the training dataset.
    points = wuhan_water.sample(**{
        'region': wuhan_ee,
        'scale': 30,
        'numPixels': 1000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })

    # Overlay the points on the imagery to get training.
    sample_point = wuhan_landsat7.select(bands).sampleRegions(**{
        'collection': points,
        'properties': [label],
        'scale': 30
    })

    # 在training要素集中增加一个random属性，值为0到1的随机数
    withRandom = sample_point.randomColumn('random')

    #把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
    split = 0.7
    trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
    testingPartition = withRandom.filter(ee.Filter.gte('random', split))

    # 利用70%数据随机森林进行训练
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(trainingPartition,label,bands)
    #对Landsat-7进行分类
    result=wuhan_landsat7.select(bands).classify(trainedClassifier)
    # Map.addLayer(result,{'min':0,'max':1,'palette': ['white','blue']},'RF_water')

    water = result.eq(1).selfMask().multiply(ee.Image.pixelArea()).divide(1e6)
    areas = water.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': wuhan_ee.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    JRC_permanent_water.append(areas.get('classification').getInfo())
    print('{} month:{}'.format(month,areas.get('classification').getInfo()))

# train_accuracy = trainedClassifier.confusionMatrix()
# print('accuracy:',train_accuracy.accuracy().getInfo())
# print('kappa:',train_accuracy.kappa().getInfo())
# print('producersAccuracy:',train_accuracy.producersAccuracy().getInfo())
# print('consumersAccuracy:',train_accuracy.consumersAccuracy().getInfo())

# validated = testingPartition.classify(trainedClassifier)

# test_accuracy = validated.errorMatrix('waterclass', 'classification')

# print('accuracy:',test_accuracy.accuracy().getInfo())
# print('kappa:',test_accuracy.kappa().getInfo())
# print('producersAccuracy:',test_accuracy.producersAccuracy().getInfo())
# print('consumersAccuracy:',test_accuracy.consumersAccuracy().getInfo())

8 month:708.1131033038536
9 month:551.551524926372
10 month:229.69852394552908
11 month:999.1459587049044
12 month:1007.625631664296


# 结果

In [25]:
print(JRC_permanent_water)

[1919.497917566903, 172.6789292320062, 178.78891883924243, 462.3357181844747, 803.3744604714041, 954.7123528474002, 817.0491499024548, 1039.7974312507413, 1126.3098211173071]


使用JRC数据集验证

In [ ]:
JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/{}".format(Year)).clip(wuhan_ee)
JRC_water = JRC_wuhan_water.gt(2).select("waterClass").rename('waterclass')
JRC_points = JRC_water.sample(**{
    'region': wuhan_ee,
    'scale': 30,
    'numPixels': 3000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

# print(points.first().getInfo())
# Overlay the points on the imagery to get training.
JRC_sample_point = wuhan_landsat8.select(bands).sampleRegions(**{
  'collection': JRC_points,
  'properties': [label],
  'scale': 30
})
print(withRandom.first().getInfo())
validated = JRC_sample_point.classify(trainedClassifier)

test_accuracy = validated.errorMatrix('waterclass', 'classification')
print('accuracy:',test_accuracy.accuracy().getInfo())
print('kappa:',test_accuracy.kappa().getInfo())
print('producersAccuracy:',test_accuracy.producersAccuracy().getInfo())
print('consumersAccuracy:',test_accuracy.consumersAccuracy().getInfo())

In [ ]:
# RF_permanent_water = []
# # ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# label = 'waterclass'
# for i in range(2010,2019,1):
#     landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#         .filterBounds(wuhan_ee) \
#         .filterDate('{}-01-01'.format(i), '{}-12-31'.format(i)) 
#     landsat_images = landsat.map(maskL8sr).map(NDVI).map(mNDWI)
#     #合并
#     wuhan_landsat8 = landsat_images.qualityMosaic('NDVI').clip(wuhan_ee)
#     # 当mNDWI > EVI时
#     wuhan_mNDWI_NDVI = wuhan_landsat8.select('mNDWI').gt(wuhan_landsat8.select('NDVI'))
#     wuhan_water = wuhan_mNDWI_NDVI.select('mNDWI').rename('waterclass')
#     points = wuhan_water.sample(**{
#         'region': wuhan_ee,
#         'scale': 30,
#         'numPixels': 1000,
#         'seed': 0,
#         'geometries': True  # Set this to False to ignore geometries
#     })
#     # Overlay the points on the imagery to get training.
#     sample_point = wuhan_landsat8.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']).sampleRegions(**{
#         'collection': points,
#         'properties': [label],
#         'scale': 30
#     })
#     # 在training要素集中增加一个random属性，值为0到1的随机数
#     withRandom = sample_point.randomColumn('random')
#     #把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
#     split = 0.7
#     trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
#     testingPartition = withRandom.filter(ee.Filter.gte('random', split))
#     # 利用70%数据随机森林进行训练
#     trainedClassifier = ee.Classifier.smileRandomForest(20).train(trainingPartition,label,['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
#     result=wuhan_landsat8.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']).classify(trainedClassifier)
#     Map.addLayer(result,{'min':0,'max':1,'palette': ['white','blue']},'RF_water{}'.format(i))
    
#     water = result.eq(1).selfMask().multiply(ee.Image.pixelArea()).divide(1e6)
#     areas = water.reduceRegion(**{
#         'reducer': ee.Reducer.sum(),
#         'geometry': wuhan_ee.geometry(),
#         'scale': 30,
#         'maxPixels': 1e14
#     })
#     RF_permanent_water.append(areas.get('waterClass').getInfo())
    
# #     print('{}year:{}'.format(i,areas.getInfo()))

In [ ]:
# # 当mNDWI > EVI时
# wuhan18_mNDWI_NDVI = wuhan18_landsat8.select('mNDWI').gt(wuhan18_landsat8.select('NDVI'))
# wuhan18_waterclass = wuhan18_mNDWI_NDVI.select('mNDWI').rename('waterclass')
# wuhan18_composition = wuhan18_landsat8.addBands(wuhan18_waterclass)
# # Map.addLayer(wuhan18_water,{'palette': 'white,blue'},'wuhan18_water')
# print(wuhan18_composition.bandNames().getInfo())

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

In [ ]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())